In [3]:
import pandas as pd
import random
from datetime import timedelta

# Load data
trade_df = pd.read_csv("Trade_table.csv")
settlement_df = pd.read_csv("settlement_instructions.csv")
security_df = pd.read_csv("seed_security_data.csv")
company_df = pd.read_csv("company_details.csv")

# Create mapping dictionaries
settlement_map = settlement_df.set_index('settlement_id').to_dict(orient='index')
security_map = security_df.set_index('security_id')['isin'].to_dict()
company_name_map = dict(zip(company_df['company_id'], company_df['company_name']))

# Custodian list
custodians = ['State Street', 'BNY Mellon', 'Northern Trust', 'HDFC Custodian']

# Broker data generation
broker_records = []

for _, trade in trade_df.iterrows():
    settlement_id = trade['settlement_id']
    settlement_info = settlement_map.get(settlement_id, {})
    
    # Extract security_id and then isin
    security_id = settlement_info.get('security_id')
    isin = security_map.get(security_id, 'UNKNOWN')

    settlement_date = pd.to_datetime(settlement_info.get('settlement_date', pd.NaT))
    trade_date = settlement_date - timedelta(days=1) if pd.notnull(settlement_date) else pd.NaT

    # Choose a broker_id (sender or receiver)
    broker_id = random.choice([settlement_info.get('sender_id'), settlement_info.get('receiver_id')])
    source = company_name_map.get(broker_id, 'UNKNOWN')

    # Determine side based on type
    side = 'Buy' if str(trade['type']).lower() == 'long' else 'Sell'

    broker_records.append({
        'trade_id': trade['trade_id'],
        'isin': isin,
        'quantity': trade['quantity'],
        'price': trade['price'],
        'trade_date': trade_date,
        'settlement_date': settlement_date,
        'counterparty': random.choice(custodians),
        'side': side,
        'created_at': trade['created_at'],
        'broker_id': broker_id,
        'source': source
    })

# Create and save broker_df
broker_df = pd.DataFrame(broker_records)

broker_df.head(10)

,trade_id,isin,quantity,price,trade_date,settlement_date,counterparty,side,created_at,broker_id,source
0,TRD25269,UNKNOWN,767,321.46,2025-06-24,2025-06-25,State Street,Buy,2025-06-23 00:52:50.028799,msly3284,Morgan Stanely
1,TRD28907,UNKNOWN,291,308.70,2025-06-24,2025-06-25,State Street,Sell,2025-06-23 20:00:42.028799,jpmc7624,JP Morgan
2,TRD87356,UNKNOWN,285,262.03,2025-06-24,2025-06-25,HDFC Custodian,Buy,2025-06-25 05:12:39.028799,bnym4392,BNY Mellon
3,TRD90371,UNKNOWN,582,247.97,2025-06-24,2025-06-25,BNY Mellon,Buy,2025-06-25 03:41:34.028799,jpmc7624,JP Morgan
4,TRD47128,UNKNOWN,799,70.40,2025-06-24,2025-06-25,HDFC Custodian,Sell,2025-06-24 14:54:51.028799,bnym4392,BNY Mellon
5,TRD50501,UNKNOWN,199,106.44,2025-06-24,2025-06-25,Northern Trust,Buy,2025-06-25 21:01:26.028799,bnpp7891,BNP Paribas
6,TRD10749,UNKNOWN,130,295.70,2025-06-24,2025-06-25,Northern Trust,Sell,2025-06-24 00:54:40.028799,gsch9157,Goldman Scahs
7,TRD37800,UNKNOWN,262,55.75,2025-06-24,2025-06-25,Northern Trust,Buy,2025-06-24 16:31:47.028799,bnym4392,BNY Mellon
8,TRD22138,UNKNOWN,651,426.76,2025-06-24,2025-06-25,Northern Trust,Sell,2025-06-24 20:52:36.028799,msly3284,Morgan Stanely
9,TRD95362,UNKNOWN,476,480.09,2025-06-24,2025-06-25,State Street,Buy,2025-06-23 03:48:20.028799,msly3284,Morgan Stanely


In [4]:
broker_df.to_csv("broker_data.csv", index=False)
